In [ ]:
import sys
from pathlib import Path

# Aggiungere la directory src al path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

import torch
from models import UNet
from utils import load_pretrained_model, resume_training

## Metodo 1: Caricare l'esperimento più recente

Specifica solo il nome del modello e il tipo di degradazione, e verrà caricato automaticamente l'esperimento più recente.

In [ ]:
# Creare il modello
model = UNet(in_channels=3, out_channels=3)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Caricare l'esperimento più recente per unet/gaussian
checkpoint_info = load_pretrained_model(
    model=model,
    experiment_path="latest",
    model_name="unet",
    degradation="gaussian",
    device=device
)

print(f"\nModello caricato dall'epoca: {checkpoint_info['epoch']}")
print(f"Metriche di validazione: {checkpoint_info['metrics'].get('val', {})}")

## Metodo 2: Caricare un esperimento specifico per timestamp

Se conosci il timestamp dell'esperimento che vuoi caricare:

In [ ]:
# Creare il modello
model = UNet(in_channels=3, out_channels=3)
model = model.to(device)

# Caricare un esperimento specifico
checkpoint_info = load_pretrained_model(
    model=model,
    experiment_path="20251229_224726",  # Timestamp dell'esperimento
    model_name="unet",
    degradation="gaussian",
    device=device
)

print(f"\nModello caricato dall'epoca: {checkpoint_info['epoch']}")

## Metodo 3: Caricare da path completo

Puoi anche specificare il path completo dall'interno di experiments:

In [ ]:
# Creare il modello
model = UNet(in_channels=3, out_channels=3)
model = model.to(device)

# Caricare specificando il path completo (relativo alla root del progetto)
checkpoint_info = load_pretrained_model(
    model=model,
    experiment_path="unet/gaussian/20251229_224726",  # Path dalla cartella experiments
    device=device
)

print(f"\nModello caricato dall'epoca: {checkpoint_info['epoch']}")

## Continuare l'addestramento da un checkpoint

Se vuoi continuare l'addestramento, puoi caricare anche l'optimizer e lo scheduler:

In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

# Creare modello, optimizer e scheduler
model = UNet(in_channels=3, out_channels=3).to(device)
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=50)

# Caricare il checkpoint completo
checkpoint_info = load_pretrained_model(
    model=model,
    experiment_path="latest",
    model_name="unet",
    degradation="gaussian",
    optimizer=optimizer,
    scheduler=scheduler,
    device=device
)

# L'addestramento può continuare dall'epoca successiva
start_epoch = checkpoint_info['epoch'] + 1
print(f"\nAddestramento riprenderà dall'epoca: {start_epoch}")

## Test del modello caricato

Verifichiamo che il modello funzioni correttamente:

## Metodo CONSIGLIATO: Resume Training (funzione wrapper)

Se vuoi riprendere l'addestramento, usa la funzione `resume_training` che carica automaticamente tutto e restituisce l'epoca da cui riprendere:

In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

# Creare modello, optimizer e scheduler (come per un nuovo training)
model = UNet(in_channels=3, out_channels=3).to(device)
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=50)

# Usa resume_training per caricare tutto automaticamente
checkpoint_info, start_epoch = resume_training(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    experiment_path="latest",  # o timestamp specifico
    model_name="unet",
    degradation="gaussian",
    device=device
)

print(f"\n🚀 Pronto per riprendere il training!")
print(f"   Start epoch: {start_epoch}")
print(f"   Previous loss: {checkpoint_info['metrics']['val']['loss']:.4f}")

# Ora puoi continuare il training normalmente
# for epoch in range(start_epoch, num_epochs):
#     train_epoch(...)

In [ ]:
# Test con un'immagine casuale
test_input = torch.randn(1, 3, 256, 256).to(device)

model.eval()
with torch.no_grad():
    output = model(test_input)

print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print("✅ Modello caricato e funzionante!")